In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pylab as plt
%matplotlib inline

from tyssue.core.sheet import Sheet

from tyssue import config
from tyssue.topology.sheet_topology import cell_division

from tyssue.geometry.sheet_geometry import SheetGeometry as geom
from tyssue.solvers.sheet_vertex_solver import Solver as solver
from tyssue.dynamics.sheet_vertex_model import SheetModel as model

from tyssue.core.generation import extrude
from tyssue.core.monolayer import Monolayer, MonolayerWithLamina
from tyssue.geometry.bulk_geometry import BulkGeometry, MonoLayerGeometry
from tyssue.dynamics.bulk_model import BulkModel, LaminaModel, set_model
from tyssue.solvers.sheet_vertex_solver import Solver

from tyssue.draw.vispy_draw import vp_view

from tyssue.draw.plt_draw import sheet_view

from tyssue.topology.sheet_topology import remove_face
from tyssue.core.objects import get_opposite
from tyssue.io.hdf5 import save_datasets, load_datasets

from tyssue.topology.sheet_topology import face_division, get_division_edges


In [18]:
datasets = load_datasets('small_ellipsoid.hf5',
                         data_names=['vert', 'edge',
                                     'face', 'cell'])
specs = config.geometry.bulk_spec()

mono = Monolayer('ell', datasets, specs)
mono.reset_index()
mono.reset_topo()


# ## Settings for the quasi-static model
# ### Apical layer
apical_spec = {
    'edge': {
        'line_tension': 0.12,},
    'face': {
        'contractility': 0.04,
        'prefered_area': 0.8,
        'area_elasticity': 1.0},
    'cell': {
        'prefered_vol': 1.0,
        'vol_elasticity': 1.0},
    'settings': {
        'grad_norm_factor': 1.0,
        'nrj_norm_factor': 1.0},}

# ### Modifier for each segment of the stratum
modifiers = {
    'sagittal': {
        'edge': {
            'line_tension': 0.1},
        'face': {
            'contractility': 0.05,
            'prefered_area': 0.2}
    },
    'basal': {
        'edge': {
            'line_tension': 0.5},
        'face': {
            'contractility': 0.1,
            'prefered_area': 0.2}},
    }

set_model(mono, BulkModel, apical_spec, modifiers)





In [19]:
draw_specs = config.draw.sheet_spec()
canvas, view = vp_view(mono, **draw_specs)

In [20]:
res = Solver.find_energy_min(mono, BulkGeometry, BulkModel)

In [21]:
canvas, view = vp_view(mono)
monolayer = mono

In [34]:
mother = 15
mother_edges = monolayer.edge_df[
    monolayer.edge_df['cell'] == mother]
mother_faces = set(mother_edges['face'])

apical_face, = mother_edges[
    mother_edges['segment'] == 'apical']['face'].unique()
basal_face, = mother_edges[
    mother_edges['segment'] == 'basal']['face'].unique()


angle = np.random.random() * np.pi
edge_aa, edge_ab = get_division_edges(monolayer, apical_face,
                                      BulkGeometry,
                                      angle=angle, axis='x')
edge_ba, edge_bb = get_division_edges(monolayer, basal_face,
                                      BulkGeometry,
                                      angle=angle, axis='x')

srce_aa, trgt_aa = mono.edge_df.loc[edge_aa, ['srce', 'trgt']]
srce_ab, trgt_ab = mono.edge_df.loc[edge_ab, ['srce', 'trgt']]

srce_ba, trgt_ba = mono.edge_df.loc[edge_ba, ['srce', 'trgt']]
srce_bb, trgt_bb = mono.edge_df.loc[edge_bb, ['srce', 'trgt']]


def get_edges(edge_df, srce, trgt):
    edges = edge_df[(edge_df['srce'] == srce) &
                    (edge_df['trgt'] == trgt)]
    if edges.shape[0] == 1:
        return edges.index[0]
    else:
        return edges.index.values


In [35]:
edges_aa = get_edges(monolayer.edge_df,
                     srce_aa, trgt_aa)
opp_edges_aa = get_edges(monolayer.edge_df,
                        trgt_aa, srce_aa)

edges_ab = get_edges(monolayer.edge_df,
                     srce_ab, trgt_ab)
opp_edges_ab = get_edges(monolayer.edge_df,
                         trgt_ab, srce_ab)

edges_ba = get_edges(monolayer.edge_df,
                     srce_ba, trgt_ba)
opp_edges_ba = get_edges(monolayer.edge_df,
                        trgt_ba, srce_ba)

edges_bb = get_edges(monolayer.edge_df,
                     srce_bb, trgt_bb)
opp_edges_bb = get_edges(monolayer.edge_df,
                         trgt_bb, srce_bb)

all_edges = np.concatenate([edges_aa, opp_edges_aa,
                            edges_ab, opp_edges_ab,
                            edges_ba, opp_edges_ba,
                            edges_bb, opp_edges_bb])
print(all_edges)

[  59  804   63  788  265 1608  264 1612  333 1458  502  782  334  698  312
  786]


In [37]:
monolayer.edge_df.loc[all_edges].sort_values(['face'])

,srce,trgt,face,segment,cell,nz,dy,dx,ny,nx,dz,sub_area,length,sub_vol,prefered_length,length_eleasticity,line_tension
edge,,,,,,,,,,,,,,,,,
59,24,19,15,apical,15,-0.052023,0.133117,0.245382,0.071283,-0.004489,0.161229,0.044181,0.322377,0.003418,0.0,0.0,0.120
265,66,88,15,apical,15,-0.046537,-0.135455,-0.200833,0.048334,-0.010480,-0.095457,0.033955,0.260373,0.002624,0.0,0.0,0.120
63,19,24,16,apical,16,-0.019217,-0.133117,-0.245382,0.079843,-0.030687,-0.161229,0.043834,0.322377,0.003060,0.0,0.0,0.120
264,88,66,56,apical,56,-0.052075,0.135455,0.200833,0.008327,0.019136,0.095457,0.028051,0.260373,0.002396,0.0,0.0,0.120
312,111,108,73,basal,9,0.003853,-0.049661,0.142352,-0.020583,-0.005726,-0.053723,0.010855,0.160052,0.000789,0.0,0.0,0.060
333,160,180,79,basal,15,0.009469,-0.012221,0.129746,-0.010694,-0.000664,-0.004702,0.007150,0.130405,0.000565,0.0,0.0,0.060
334,108,111,79,basal,15,0.011476,0.049661,-0.142352,-0.013716,-0.000454,0.053723,0.008945,0.160052,0.000704,0.0,0.0,0.060
502,180,160,126,basal,62,0.012104,0.012221,-0.129746,-0.002331,0.000219,0.004702,0.006164,0.130405,0.000518,0.0,0.0,0.060
698,108,111,164,sagittal,9,-0.031955,0.049661,-0.142352,-0.014201,-0.017014,0.053723,0.019444,0.160052,0.001198,0.0,0.0,0.012


In [13]:
sagittal_edges = mother_edges[
    mother_edges['segment'] == 'sagittal']

sagittal_edges[(sagittal_edges['trgt'] == srce_a) &
               (sagittal_edges['srce'] == trgt_a)].shape[0]

1

In [15]:
sagittal_edges = mother_edges[
    mother_edges['segment'] == 'sagittal']
sagittal_faces = sagittal_edges['face'].unique()

s_edge_aa = sagittal_edges[(sagittal_edges['trgt'] == srce_a) &
                           (sagittal_edges['srce'] == trgt_a)].index[0]
s_edge_ab = sagittal_edges[(sagittal_edges['trgt'] == srce_b) &
                           (sagittal_edges['srce'] == trgt_b)].index[0]

s_face_a = int(monolayer.edge_df.loc[s_edge_aa,
                                     'face'])

s_face_b = int(monolayer.edge_df.loc[s_edge_ab,
                                     'face'])

opp_s_face_a = opposite_face(monolayer, s_face_a)
opp_s_edges_a = monolayer.edge_df[
    monolayer.edge_df['face'] == opp_s_face_a]
opp_s_edge_aa = 
opp_s_face_b = opposite_face(monolayer, s_face_b)



In [23]:
sagittal_edges.loc[s_edge_a, 'face']

188

In [8]:
def get_basal_edge(monolayer, sagittal_edge):
    face = int(monolayer.edge_df.loc[sagittal_edge, 'face'])
    edges = monolayer.edge_df[
        monolayer.edge_df['face'] == face]
    srce_segments = monolayer.vert_df.loc[edges['srce'], 'segment']
    srce_segments.index = edges.index
    trgt_segments = monolayer.vert_df.loc[edges['trgt'], 'segment']
    trgt_segments.index = edges.index

    basal_edge, = edges[(srce_segments == 'basal') &
                        (trgt_segments == 'basal')].index
    return basal_edge


def opposite_face(eptm, face):
    vert_orbit = eptm.get_orbits('face', 'srce').groupby(
        level='face').apply(lambda df: set(df))
    print(vert_orbit.loc[face])
    opp_faces = vert_orbit[vert_orbit == vert_orbit.loc[face]].index
    print(opp_faces)
    if len(opp_faces) == 2:
        opp_face, = opp_faces[opp_faces != face]
        return opp_face
    else:
        return None

    
apical_daughter = face_division(monolayer, edge_a, edge_b)

In [28]:
%pdb

Automatic pdb calling has been turned ON


In [37]:
s_face_a = int(monolayer.edge_df.loc[])


b_edge_a = get_basal_edge(monolayer, s_edge_a)
b_edge_b = get_basal_edge(monolayer, s_edge_b)



sagittal_daughter_a = face_division(monolayer,
                                    s_edge_a,
                                    b_edge_a)
sagittal_daughter_b = face_division(monolayer,
                                    s_edge_b,
                                    b_edge_b)



In [38]:
canvas, view = vp_view(mono)

In [52]:
vert_orbit = monolayer.get_orbits('face', 'srce').groupby(
    level='face').apply(lambda df: set(df))


In [58]:
vo = vert_orbit.loc[mono.sagittal_faces].head(10)
vo[vo == vo.loc[132]]

face
132    {1, 2, 93, 94}
133    {1, 2, 93, 94}
Name: srce, dtype: object

In [47]:
sagittal_faces

array([184, 185, 186, 187, 188, 393])

In [60]:
opposite_face(monolayer, sagittal_faces[0])

{16, 188, 108, 189}
Int64Index([184], dtype='int64', name='face')


In [61]:
fv = {16, 188, 108, 189}

vert_orbit[vert_orbit == fv]

face
184    {16, 188, 108, 189}
Name: srce, dtype: object

In [51]:
{64, 0, 2, 78, 1} == {64, 1, 2, 78, 0}

True